In [2]:
import sys

%load_ext autoreload
%autoreload 2
    
sys.path.append('../scripts/')
from entities import *
from commands import *


In [3]:
book = parse_book('../../datasets/Gutenberg/txt/Charles Dickens___A Christmas Carol.txt', verbose=True, \
              threads=8, batch_size=8)

===================Begin Parsing======================
Sentences and Coref obtained : 9.523380279541016
Emotions obtained : 7.794719219207764

Sentences : 2529, characters : 161
Processing_time : 17.318172216415405
===================End Parsing======================


In [24]:
import bz2
import _pickle as cPickle
book_dict = book.to_dict()

with bz2.BZ2File('res.pbz2', 'w') as f: 
    cPickle.dump(book_dict, f)

with bz2.BZ2File('res.pbz2', 'r') as f: 
    book_recovered = cPickle.load(f)

assert book_dict==book_recovered